In [1]:
import pandas as pd
import string
import nltk

In [2]:
from nltk.corpus import stopwords

In [3]:
st = stopwords.words('english')
st.append("and")

In [4]:
products = pd.read_csv('amazon_baby.csv')

In [5]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [6]:
def remove_punctuation(message):
    test_punc_removed = [char for char in str(message) if char not in string.punctuation]
    test_punc_removed = ''.join(test_punc_removed)
    test_punc_st_removed = []
    for char in test_punc_removed.split():
        if char.lower() not in st:
            test_punc_st_removed.append(char)
    test_punc_st_removed = " ".join(test_punc_st_removed)  
    return test_punc_st_removed

In [7]:
products['review_clean'] = products['review'].apply(remove_punctuation)

In [8]:
products = products.fillna({'review':''})

In [9]:
products = products[products['rating'] != 3] #remove neutrals

In [10]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X = products.drop('sentiment', axis = 1)
y = products['sentiment']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b') # Use this token pattern to keep single-letter words

X_train_matrix = vectorizer.fit_transform(X_train['review_clean'])
X_test_matrix = vectorizer.transform(X_test['review_clean'])

In [15]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter = 10000)
logreg.fit(X_train_matrix, y_train)

LogisticRegression(max_iter=10000)

In [16]:
len(logreg.coef_[logreg.coef_>=0])

90851

In [17]:
sample_test_data = X_test[15:25]
print(sample_test_data) 

                                                     name  \
167897  Wrapsody Breeze Baby Carrier, Freya, Medium/Large   
129689                  Noah And Friends High Chair Cover   
54150              Medela Pump in Style Advanced Backpack   
8352                            Baby Bjorn Toilet Trainer   
55700                       HABA Blossom Butterfly Mobile   
83479           Dreambaby Super Toy Hammock and Toy Chain   
93618                Sunshine Kids Seat Belt Pillow, Grey   
130762            Dreambaby Sliding Door and Window Locks   
73991   Delta Canton 4-in-1 Convertible Crib, Dark Cherry   
174021  Thermos FUNtainer Monsters University Food Jar...   

                                                   review  rating  \
167897  This wrap is very thin and lightweight, great ...       5   
129689                                                          5   
54150   I got this pump as I went back to work with my...       5   
8352    Before I had purchased this toilet trainer, 

In [18]:
sample_test_data['review_clean']

167897    wrap thin lightweight great summertime versati...
129689                                                  nan
54150     got pump went back work second child rented pu...
8352      purchased toilet trainer using cheaper seat so...
55700     son 6 weeks old bought 4 weeks hung changing t...
83479     Unfortunately like cement high rise cant screw...
93618     purchased husband bypass surgery works great l...
130762    product expensive tried use sliding door follo...
73991     Really nice looking easy put togetherLooks rea...
174021    thermos great keeping food warm fall character...
Name: review_clean, dtype: object

In [19]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = logreg.decision_function(sample_test_matrix)
print(scores)

[ 2.60158494  1.87912477  0.02771336  7.11899316  5.17118482  1.25003051
  6.63318809 -1.80259694  5.35936928  2.36601912]


In [20]:
y_pred = logreg.predict(X_test_matrix)

In [21]:
from sklearn.metrics import accuracy_score

In [22]:
ac = accuracy_score(y_test, y_pred)

In [23]:
ac

0.9238703487151809

In [24]:
probs = 1/(1+2.71828**(-1*scores))

In [25]:
probs

array([0.9309634 , 0.86751042, 0.50692789, 0.99919107, 0.9943542 ,
       0.777305  , 0.99868576, 0.14153538, 0.99531813, 0.91419899])